## Agent 란?

에이전트 프레임워크를 가장 잘 이해하기 위해서, 온라인에서 정보를 찾는 도구 하나와 우리가 인덱스로 로드한 특정 데이터를 찾는 도구 하나를 가진 에이전트를 만들어 보겠습니다.

### 단계별 추적을 위한 LangSmith 설정

LangChain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용하게 됩니다.

단계별로 호출되는 로직이 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요해집니다. 이를 위한 최선의 방법은 [LangSmith](https://smith.langchain.com)를 사용하는 것입니다.

LangSmith가 필수는 아니지만, 매우 유용한 도구입니다. LangSmith를 사용하고 싶다면, 위의 링크에서 가입한 후, 로깅 추적을 시작하기 위해 환경 변수를 설정해야 합니다.

## 도구(Tools)

Agent 가 활용할 도구를 정의하여 Agent 가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있습니다.

Tavily Search 는 그 중 대표적인 **검색 도구** 입니다. 검색을 통해 최신 정보에 접근하여 검색 결과를 가지고 답변을 생성할 수 있습니다. 도구는 이처럼 검색 도구 뿐만아니라 Python 코드를 실행할 수 있는 도구, 직접 정의한 함수를 실행하는 도구 등 다양한 종류와 방법론을 제공합니다.


### 검색도구: Tavily Search

LangChain에는 Tavily 검색 엔진을 도구로 쉽게 사용할 수 있는 내장 도구가 있습니다.

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아야 합니다.

- [Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록 합니다.

아래 코드의 주석을 풀고 발급받은 API KEY 를 설정합니다.


In [ ]:
# import os

# os.environ["TAVILY_API_KEY"] = "TAVILY API KEY 입력"

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [ ]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=5)

`search.invoke` 함수는 주어진 문자열에 대한 검색을 실행합니다.

`invoke()` 함수에 검색하고 싶은 검색어를 넣어 검색을 수행합니다.


In [ ]:
# 검색 결과를 가져옵니다.
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

### 도구: Retriever

우리의 데이터에 대해 조회를 수행할 retriever도 생성합니다.


이 코드는 웹 기반 문서 로더, 문서 분할기, 벡터 저장소, 그리고 OpenAI 임베딩을 사용하여 문서 검색 시스템을 구축합니다.

여기서는 PDF 문서를 `FAISS` DB 에 저장하고 조회하는 retriever 를 생성합니다.


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

이 함수는 `retriever` 객체의 `get_relevant_documents` 메소드를 사용하여 "YouTube의 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 알려줘" 와 관련된 문서들 중 첫 번째 문서를 반환합니다.

이는 특정 쿼리에 대한 가장 **관련성 높은 문서** 를 찾는 데 사용됩니다.


In [ ]:
# PDf 문서에서 Query 에 대한 관련성 높은 Chunk 를 가져옵니다.
retriever.get_relevant_documents(
    "YouTube의 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 알려줘"
)[0]

이제 우리가 검색을 수행할 인덱스를 채웠으므로, 이를 에이전트가 제대로 사용할 수 있는 도구로 쉽게 변환할 수 있습니다.


`create_retriever_tool` 함수는 `langchain` 라이브러리의 `tools.retriever` 모듈에서 가져온다. 이 함수는 특정 데이터를 검색하기 위한 도구를 생성하는 데 사용된다. `langchain`은 언어 모델과 관련된 다양한 기능을 제공하는 라이브러리로, 이 중 검색 도구 생성 기능은 데이터 검색 및 처리 작업을 용이하게 한다.


In [ ]:
# langchain 패키지의 tools 모듈에서 retriever 도구를 생성하는 함수를 가져옵니다.
from langchain.tools.retriever import create_retriever_tool

이 함수는 `create_retriever_tool`을 호출하여 `retriever_tool`을 생성합니다. 이 도구는 `LangSmith`에 대한 정보를 검색하는 데 사용됩니다.

사용자가 `LangSmith`에 대해 궁금한 점이 있을 때, 이 도구를 통해 질문을 해결할 수 있습니다. 함수는 `retriever` 인스턴스, 검색 도구의 이름인 `langsmith_search`, 그리고 사용자에게 도구 사용 방법을 안내하는 메시지를 인자로 받습니다.


In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="2023년 12월 AI 관련 정보를 PDF 문서에서 검색합니다. '2023년 12월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

### 사용할 도구를 정의

이제 두 가지를 모두 만들었으므로, Agent 가 사용할 도구 목록을 만들 수 있습니다.


`tools` 리스트는 `search`와 `retriever_tool`을 포함합니다. 이 리스트는 검색 및 정보 검색 도구를 저장하는 데 사용됩니다. 각 요소는 특정 작업을 수행하는 데 필요한 기능을 제공합니다.


In [ ]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

## 에이전트 생성

이제 도구를 정의했으니 에이전트를 생성할 수 있습니다. OpenAI Functions 에이전트를 사용할 것입니다.

먼저, 에이전트가 활용할 LLM을 정의합니다.


`ChatOpenAI` 클래스는 `langchain_openai` 모듈에서 가져온 것으로, OpenAI의 언어 모델을 활용하여 대화형 AI를 구현할 수 있게 해줍니다.

이 예제에서는 `gpt-3.5-turbo` 모델을 사용하며, `temperature` 매개변수를 `0`으로 설정하여 예측의 변동성을 최소화합니다.


In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 클래스를 langchain_openai 모듈에서 가져옵니다.
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

다음으로, 에이전트를 안내하기 위해 사용하고 싶은 프롬프트를 선택합니다.

이 프롬프트의 내용을 보고 싶고 LangSmith에 접근할 수 있다면, 다음으로 이동하세요:

다운로드 받아 활용할 프롬프트: https://smith.langchain.com/hub/hwchase17/openai-functions-agent


`hub.pull()` 함수를 사용하여 리소스를 가져온 후, `prompt.messages`를 통해 해당 리소스의 메시지에 접근합니다.


In [ ]:
from langchain import hub

# hub에서 prompt를 가져옵니다 - 이 부분을 수정할 수 있습니다!
prompt = hub.pull("hwchase17/openai-functions-agent")

# prompt 의 messages를 출력합니다.
prompt.messages

이제 LLM, 프롬프트 및 도구로 에이전트를 초기화할 수 있습니다. 에이전트는 입력을 받아 어떤 `Action` 을 취할지 결정하는 역할을 합니다.

중요한 것은 에이전트가 `Action` 들을 실행하지 않는다는 것입니다 - 그것은 AgentExecutor(다음 단계)에 의해 수행됩니다.


In [ ]:
from langchain.agents import create_openai_functions_agent

# OpenAI 함수 기반 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_openai_functions_agent(llm, tools, prompt)

마지막으로, 우리는 에이전트(`agent`)를 AgentExecutor 내부의 도구들과 결합합니다(이는 반복적으로 에이전트를 호출하고 도구들을 실행할 것입니다).


이 코드는 `langchain.agents` 모듈에서 `AgentExecutor` 클래스를 가져와 인스턴스를 생성합니다. 생성 시, `agent`, `tools` 객체를 인자로 전달하고, `verbose=True`를 설정하여 상세한 로그 출력을 활성화합니다. `AgentExecutor`는 주어진 에이전트와 도구들을 사용하여 작업을 실행하는 역할을 합니다.


In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 에이전트 실행하기

이제 몇 가지 질의에 대해 에이전트를 실행할 수 있습니다!

현재 이러한 모든 질의는 **상태(Stateless) 가 없는** 질의입니다(이전 상호작용을 기억하지 않습니다).


`agent_executor` 객체의 `invoke` 메소드는 딕셔너리 형태의 인자를 받아 처리합니다. 이 예제에서는 `input` 키에 `hi!` 값을 할당한 딕셔너리를 인자로 전달하고 있습니다. 이는 일반적으로 AI 에이전트, 함수 실행기, 또는 명령 처리기 등의 객체에서 입력을 처리하기 위해 사용됩니다.


In [ ]:
# 'agent_executor' 객체의 'invoke' 메소드를 호출하여,
# 'input' 키와 '안녕, 반가워' 값을 가진 딕셔너리를 인자로 전달합니다.
response = agent_executor.invoke({"input": "안녕, 반가워!"})
print(f'답변: {response["output"]}')

`agent_executor` 객체의 `invoke` 메소드를 사용하여, 질문을 입력으로 제공합니다.


In [ ]:
# 'agent_executor' 객체의 'invoke' 메소드를 호출하여, 'langsmith'가 테스팅에 어떻게 도움을 줄 수 있는지에 대한 질문을 입력으로 제공합니다.
response = agent_executor.invoke(
    {
        "input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 PDF 문서에서 알려줘"
    }
)
print(f'답변: {response["output"]}')

In [ ]:
# 검색 결과를 요청 후 질문에 대한 답변을 출력합니다.
response = agent_executor.invoke(
    {"input": "판교 카카오 프렌즈샵 아지트점의 전화번호를 검색하여 결과를 알려주세요."}
)
print(f'답변: {response["output"]}')

## 메모리 추가하기

앞서 언급했듯이, 이 에이전트는 상태가 없습니다. 이는 이전 상호작용을 기억하지 않는다는 것을 의미합니다. 이에게 메모리를 부여하기 위해서는 이전 `chat_history`를 전달해야 합니다.

**[참고]**

우리가 사용하는 프롬프트 때문에 `chat_history`라고 불러야 합니다. 다른 프롬프트를 사용한다면 **변수 이름을 변경**할 수 있습니다.


함수는 사용자의 입력(`input`)과 채팅 기록(`chat_history`)을 매개변수로 받습니다. 여기서, `chat_history`는 채팅의 첫 번째 메시지를 처리할 때 빈 리스트([])로 전달됩니다. 이는 채팅 세션을 시작할 때 기존의 대화 기록이 없음을 나타냅니다.


In [ ]:
# 채팅의 첫 번째 메시지이므로 chat_history에 빈 리스트를 전달합니다.
response = agent_executor.invoke(
    {"input": "Hi! I'm Teddy. Glad to meet you.", "chat_history": []}
)
print(f'답변: {response["output"]}')

이 모듈은 `langchain_core.messages`에서 `AIMessage`와 `HumanMessage` 클래스를 가져옵니다. AI와 인간 간의 메시지 교환을 처리하는 데 사용될 수 있습니다.

`AIMessage`는 AI가 생성한 메시지를 나타내며, `HumanMessage`는 사용자가 입력한 메시지를 나타냅니다.


In [ ]:
# langchain_core.messages에서 AIMessage와 HumanMessage를 가져옵니다.
from langchain_core.messages import AIMessage, HumanMessage

이 함수는 대화 에이전트를 실행하여 사용자와 AI 간의 대화 이력과 새로운 사용자 입력을 처리합니다. `agent_executor.invoke` 메소드는 대화 이력(`chat_history`)과 사용자의 새로운 입력(`input`)을 매개변수로 받습니다. `chat_history`는 `HumanMessage`와 `AIMessage` 객체의 리스트로, 각각 사용자와 AI의 메시지를 나타냅니다. 이 예시에서는 사용자가 "hi! my name is bob"이라고 인사하고, AI가 "Hello Bob! How can I assist you today?"라고 응답한 이후, 사용자가 "what's my name?"이라고 새로운 질문을 합니다.


In [ ]:
response = agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="Hi! I'm Teddy. Glad to meet you."),
            AIMessage(
                content="Hello Teddy! I'm glad to meet you too. How can I assist you today?"
            ),
        ],
        "input": "What's my name?",
    }
)
print(f'답변: {response["output"]}')

만약 이 메시지들을 자동으로 추적하고 싶다면, 이를 RunnableWithMessageHistory로 감싸면 됩니다.


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

`ChatMessageHistory` 클래스의 인스턴스를 생성하여 `message_history` 변수에 할당합니다. 이 인스턴스는 채팅 메시지의 기록을 관리하는 데 사용됩니다.


In [ ]:
# 채팅 메시지 기록을 관리하는 객체를 생성합니다.
message_history = ChatMessageHistory()

`RunnableWithMessageHistory` 클래스는 에이전트 실행자(`agent_executor`)와 메시지 기록을 관리합니다. 이 클래스는 세션 ID를 기반으로 메시지 기록을 처리하는 람다 함수와, 입력 메시지 및 채팅 기록의 키를 인자로 받습니다.

여기서 세션 ID는 실제로 사용되지 않으며, 단순한 메모리 내 `ChatMessageHistory`를 사용하는 예시입니다.

기본적으로 실행 가능 항목은 문자열인 `session_id`라는 단일 구성 매개변수를 받습니다.
이 매개변수는 주어진 session_id와 일치하는 새 채팅 메시지 기록을 만들거나 기존 채팅 메시지 기록을 조회하는 데 사용됩니다.

이 경우 호출은 다음과 같습니다.

`with_history.invoke(..., config={"configurable": {"session_id": "bar"}})`;

예: `{"configurable": {"session_id": "<세션_id>"}}`.


In [ ]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대부분의 실제 시나리오에서 세션 ID가 필요하기 때문에 이것이 필요합니다
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    lambda session_id: message_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

`agent_with_chat_history.invoke` 함수는 사용자 입력과 설정을 인자로 받아 처리합니다. 이 예제에서는 사용자의 입력(`"hi! I'm bob"`)과 설정(`config`)을 전달합니다. 설정에는 `session_id`가 포함되어 있으나, 이는 간단한 메모리 내 `ChatMessageHistory`를 사용하는 예제이므로 실제로는 사용되지 않습니다. 이 코드는 대부분의 실제 상황에서 세션 ID가 필요함을 보여주지만, 특정 구현에서는 그렇지 않을 수 있음을 보여줍니다.


In [ ]:
response = agent_with_chat_history.invoke(
    {"input": "Hi! I'm Teddy. Glad to meet you."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")

이 함수 호출은 `agent_with_chat_history` 객체를 사용하여 입력된 질문에 대한 응답을 생성합니다. 입력은 `input` 키를 통해 제공되며, 여기서는 "what's my name?"이 사용됩니다. 또한, `config` 매개변수를 통해 추가 설정을 제공할 수 있으며, 이 예제에서는 `session_id`를 `<foo>`로 설정합니다. 이는 대부분의 실제 시나리오에서 세션 ID가 필요하지만, 이 경우 간단한 메모리 내 `ChatMessageHistory`를 사용하기 때문에 실제로는 사용되지 않음을 나타냅니다.


In [ ]:
response = agent_with_chat_history.invoke(
    {"input": "What's my name?"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")

In [ ]:
response = agent_with_chat_history.invoke(
    {"input": "판교 카카오 프렌즈샵 아지트점의 전화번호를 검색하여 결과를 알려주세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")

In [ ]:
response = agent_with_chat_history.invoke(
    {
        "input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 PDF 문서에서 알려줘"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")

## Agent 템플릿


In [3]:
# 필요한 모듈 import
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=5은 검색 결과를 5개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=5)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

# langchain 패키지의 tools 모듈에서 retriever 도구를 생성
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    # 도구에 대한 설명을 자세히 기입해야 합니다!!!
    description="2023년 12월 AI 관련 정보를 PDF 문서에서 검색합니다. '2023년 12월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

########## 3. Prompt 를 정의합니다 ##########

# hub에서 prompt를 가져옵니다 - 이 부분을 수정할 수 있습니다!
prompt = hub.pull("hwchase17/openai-functions-agent")

########## 4. Agent 를 정의합니다 ##########

# OpenAI 함수 기반 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_openai_functions_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# 채팅 메시지 기록을 관리하는 객체를 생성합니다.
message_history = ChatMessageHistory()

# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대부분의 실제 시나리오에서 세션 ID가 필요하기 때문에 이것이 필요합니다
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    lambda session_id: message_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. 질의-응답 테스트를 수행합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.invoke(
    {
        "input": "YouTube 2024년부터 AI 생성콘텐츠 표시 의무화에 대한 내용을 PDF 문서에서 알려줘"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "MyTestSessionID"}},
)
print(f"답변: {response['output']}")



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': 'YouTube 2024년 AI 생성콘텐츠 표시 의무화'}`


SPRi AI Brief |  
2023-12 월호
14유튜브 , 2024년부터 AI 생성 콘텐츠 표시 의무화 
n유튜브가 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 라벨 표시를 의무화하기로 했으며 , 이를 
준수하지 않는 콘텐츠는 삭제하고 크리에이터에 대한 수익 배분도 중단할 수 있다고 설명
n유튜브는 AI 생성 콘텐츠가 신원 파악이 가능한 개인을 모방한 경우 개인정보 침해 신고 
절차에 따라 콘텐츠 삭제 요청도 받을 계획KEY Contents
£유튜브 , 생성 AI 콘텐츠에 AI 라벨 표시 안 하면 콘텐츠 삭제
n유튜브가 2023 년 11월 14일 공식 블로그를 통해 몇 달 안에 생성 AI를 사용한 콘텐츠에 AI 
라벨을 표시하는 새로운 규칙을 시행한다고 발표 
∙실제로 일어나지 않은 사건을 사실적으로 묘사하거나 실제로 하지 않은 말이나 행동을 보여주는 콘텐츠와 
같이 AI 도구를 사용해 사실적으로 변경되거나 합성된 콘텐츠에는 AI 라벨을 표시 필요
∙유튜브는 이러한 규칙이 선거나 분쟁 상황, 공중 보건, 공직자 관련 문제와 같이 민감한 주제를 다루는 
콘텐츠에서 특히 중요하다고 강조했으며 , 크리에이터가 AI로 제작한 콘텐츠에 AI 라벨을 표시하지 않으면 
해당 콘텐츠는 삭제되고 광고 수익을 배분하는 유튜브 파트너 프로그램도 정지될 수 있음
∙유튜브는 두 가지 방식으로 AI를 이용한 콘텐츠의 변경이나 합성 여부를 시청자에게 전달할 계획으로 
동영상 설명 패널에 라벨을 표시하는 방식이 기본이며 , 민감한 주제를 다루는 특정 유형의 콘텐츠는 동영상 
플레이어에 더욱 눈에 띄는 라벨을 적용 
∙유튜브는 커뮤니티 정책에 위반되는 일부 합성 콘텐츠에 대해서는 라벨 지정 여부와 관계없이 삭제할 
방침으로 , 가령 사실적인 폭력을 보여주는 합성 동영상이 시청자

마무리입니다!

이번 튜토리얼에서는 우리는 간단한 에이전트를 만드는 방법을 다뤘습니다.

에이전트는 복잡한 주제이며 배울 것이 많습니다! 개념 가이드, 다양한 유형의 에이전트, 맞춤형 도구를 만드는 방법 등 더 많은 자료를 찾으려면 [한국어 튜토리얼(Agent)](https://github.com/teddylee777/langchain-kr/tree/main/08-Agent) 를 참고해 주세요!
